In [1]:
import pandas as pd
import logging
import csv
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf
import re
import time
from pyspark.sql import functions as F
from pyspark.sql.functions import length, col, explode, upper, to_date, date_sub, lag, coalesce, lit, array_sort, when, arrays_zip, size, date_format, explode_outer, from_json, concat, expr, array
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.utils import AnalysisException
from operator import itemgetter
import datetime, re, requests
from pyspark.sql import Window
from pyspark.sql.functions import concat, col, lit
from pyspark.sql.functions import split
from pyspark.sql.functions import when
from pyspark.sql.functions import current_timestamp
from datetime import datetime
# import boto3
# from boto3 import client
# from boto3.dynamodb.conditions import Key
import json

In [2]:
# import sys
# from awsglue.transforms import *
# from awsglue.utils import getResolvedOptions
# from pyspark.context import SparkContext
# from awsglue.context import GlueContext
# from awsglue.job import Job
#
# ## @params: [JOB_NAME]
# args = getResolvedOptions(sys.argv, ['JOB_NAME'])
#
# sc = SparkContext()
# glueContext = GlueContext(sc)
# spark = glueContext.spark_session
# job = Job(glueContext)
# job.init(args['JOB_NAME'], args)

In [3]:
# # read data from S3 to DataFrame
# # input: S3
# # output df['ROW_ID', 'BOS_FILE_EXTRACT', 'file_path', 'COLUMN_DEF']
# def read_s3_to_df(day_path):
#
#     # parameter
#     i = 0 # flag for initialize dataframe
#
#     # boto3 client
#     s3 = boto3.client('s3')
#     conn = client('s3')  # again assumes boto.cfg setup, assume AWS S3
#
#     # read data file by file
#     for key in conn.list_objects(Bucket='bos-etl')['Contents']:
#         path_key = key['Key']
#         if path_key.endswith('cleansed'):
#             if day_path in path_key:
#                 file = s3.get_object(Bucket='bos-etl', Key=path_key)
#                 txt = (file['Body'].read().decode('latin1'))
#                 #st_re = txt.replace("", ",")
#                 st_re_newline = txt.replace("!!! EOS !!!", "\n")
#                 st_re_split = st_re_newline.split("\n")
#                 df = pd.DataFrame(st_re_split)
#                 df.index.name = 'ROW_ID'
#                 df.rename({0:'BOS_FILE_EXTRACT'},axis='columns',inplace=True)
#                 df["COLUMN_DEF"]=df['BOS_FILE_EXTRACT'].replace(regex=r"\.*",value="")
#                 # rslt_dfRFT_temp = df[df['COLUMN_DEF'] =='PAT'].head(10)
#                 rslt_dfRFT_temp = df
#                 if ~rslt_dfRFT_temp.empty:
#                     # print(path_key)
#                     rslt_dfRFT_temp.insert(0,'tax_on_commission', path_key)
#                     if (i ==0):
#                         rslt_dfRFT = rslt_dfRFT_temp
#                         i = 1
#                     else:
#                         rslt_dfRFT = pd.concat([rslt_dfRFT,rslt_dfRFT_temp])
#     return rslt_dfRFT
#
#
# # read data as DataFrame
# #select day
# day_path = 'date=2023-06-17'
# # select current system day
# # day_path = time.strftime('%Y-%m-%d')
# rslt_dfRFT = read_s3_to_df(day_path)

In [5]:
# read csv
rslt_dfRFT = pd.read_csv("source/input/1_df_RFT_PAT_PAX.csv")

# DataFrame to Dataframe
from pyspark.sql import SparkSession

#Create PySpark SparkSession
spark = SparkSession.builder\
    .master("local[1]")\
    .appName("SparkByExamples.com")\
    .getOrCreate()
#Create PySpark DataFrame from Pandas
sparkDF = spark.createDataFrame(rslt_dfRFT)

In [62]:
# Common
# coupons format
def coupons_format(s):
    coupons = 10000
    coupons_dict = {"1": 1000, "2": 200, "3": 30, "4": 4}

    for i in range(len(s)):
        if s[i] in coupons_dict:
            coupons = coupons + coupons_dict[s[i]]

    return str(coupons)[1:]

# check None
def check_None(value, name, exception):
    try:
        if value is None:
            msg = {
                name : "ValueIsNone"
            }

            # append
            exception.append(msg)

    except Exception as e:
        exception.append(str(e))

    return exception

# check Empty
def check_Empty(value, name, exception):
    try:
        if len(value) == 0:
            msg = {
                name : "ValueIsEmpty"
            }

            # append
            exception.append(msg)

    except Exception as e:
        exception.append(str(e))

    return exception

# check len
def check_len(value, name, exception, min, max):
    try:
        if (len(value) < min) & (len(value) > max):
            msg = {
                name : f"ValueOutOfRange: {value} out of ({min},{max})"
            }

            # append
            exception.append(msg)

            # value
            value = 'NULL'
    except Exception as e:
        exception.append(str(e))
        value = 'NULL'

    return value, exception

# check float
def check_float(value, name, exception, integ, demic):
    try:
        value_str = str(float(value))
        if (len(value_str.split('.')[0]) > (integ - demic)) & (len(value_str.split('.')[1]) > demic):
            msg = {
                name : f"FloatOutOfRange: {value} out of ({integ},{demic})"
            }

            # append
            exception.append(msg)

            # value
            value = -0.99
    except Exception as e:
        exception.append(str(e))
        value = -0.99

    return value, exception

# check Match
def check_Match(value1, name1, value2, name2, exception):
    try:
        if str(value1) != str(value2):
            msg = {
                f"({name1},{name2})" : f"ValueIsNotMatch: ({value1},{value2})"
            }

            # append
            exception.append(msg)

    except Exception as e:
        exception.append(str(e))

    return exception

In [63]:
# BRE Def
# commission json def
def commission_json(SAL_json, currency_code):
    ''' COMMISSION:
        #JSON. Example: [{"type":"BASE","amount":4.26,"currency":"CAD","commissionRate":3.0}]
        mount: 13th field from 2nd group (COAM-SAL)
        commissionRate: 14th field from 2nd group (CORT-SAL)'''

    # json
    commission = json.dumps({
        "type": 'BASE',
        "amount": SAL_json['COAM'],
        "currency": currency_code,
        "commissionRate": SAL_json['CORT']
    })

    return commission

# tax json def
def tax_json(TAX_L1, original_currency):
    ''' TAX:
        Field will be <null> if is RFND
        Note: this group may have a loop
        JSON. Example: [{"type":"CA","amount":7.12,"currency":"CAD"},{"type":"YR","amount":16.00,"currency":"CAD"}]
        type: 2nd field of the loop from 3rd group (TMFT-TAX)
        amount: 1st field of the loop from 3rd group (TMFA-TAX)
        currency: 10th field from 2nd group (CUOF-SAL)'''
    # json list
    tax_list = []
    for i in range(len(TAX_L1)):
        # json
        tax_dict = {
            "type": TAX_L1[i]['TMFT'],
            "amount": TAX_L1[i]['TMFA'],
            "currency": original_currency
        }
        # append list
        tax_list.append(tax_dict)

    return json.dumps(tax_list)

# leg json df
def legs_json(fare_construction, ITI_L, currency_code, original_currency, ticket_number, issue_date, exception):
    '''LEGS:
        NOTE: If the ticket type is RFND, LEGS field needs to be empty
        Note: this info is provided in the ITI group. Each ticket may have 4 legs max. After the 5th leg,the ticket is considered as conjunction. In the file, if there is a loop, the ticket has aconjunction ticket.Example: 1st leg, 2nd leg, 3rd leg, 4th leg + loop + 5th leg…If the leg is empty, it means that the ticket stopped in the previous leg. Don’t load empty values in the Json.
        JSON. example: [{"departure":"FLR","destination":"YYZ","seatClass":"C","conjunction":"1111234567890","carrier":"AC","tripCode":"876","departureOn":"2022-12-30","designator":"","stopOver":"X","flyerCode":"","fare":259.25,"currency": "CAD","originalFare":259.25,"originalCurrency":"CAD"}]
        departure: from 4th group (ORAC-ITI) → Leg 1: 4th field | Leg 2: 13th field | Leg 3: 22nd field | Leg 4: 31st field
        destination: from 4th group (DSTC-ITI) → Leg 1: 5th field | Leg 2: 14th field | Leg 3: 23rd field | Leg 4: 32nd field
        seatClass: 4th group (CLSC-ITI) → Leg 1: 8th field | Leg 2: 17th field | Leg 3: 26th field | Leg 4: 35th field
        conjunction: 1st field from 4th group (CJNR-ITI). If is empty, use the same as TICKET_NUMBER
                    if the ticket includes more than 4 legs: in ITI section, the 39th field (the field after 4th repeat’s designator) will give a new BDNR (10 digits), use the BACN from DCI + new BDNR as conjunction for the following legs
        carrier: from 4th group (CARR-ITI) → Leg 1: 6th field | Leg 2: 15th field | Leg 3: 24th field | Leg 4: 33rd field
        tripCode: 4th group (FTNR-ITI) → Leg 1: 7th field | Leg 2: 16th field | Leg 3: 25th field | Leg 4: 34th field
        departured on: 4th group (FTDA-ITI) → Leg 1: 9th field | Leg 2: 18th field | Leg 3: 27th field | Leg 4: 36th field. NOTE: you need to store this format in the databse: YYYY-MM-DD but the file has JAN01 for example. Use the same procedure/logic from CAT file loader in order to convert into date. Check the logic from CAT (Java code) with Haibinhg and Santhosh because there are some tricks in the code but the logic is:
        File will come as JUL01 (they don’t report the year) → convert into 2023-07-01
        If the departure date is before the issue date, the year will be ISSUE_DATE +1 year. Example: issue date is 2023-07-01 and departure date is JUN01, then the departure date will be 2024-06-01
         If the departure date is after or equals to the issue date, the year will be the same as Issue Date. Example: issue date is 2023-07-01 and departure date is DEC01, then the departure date will be 2023-12-01
        If the departure date is after the issue date (but after december 31st), the year will be the same as Issue Date + 1 year. Example: issue date is 2023-07-01 and departure date is JAN01, then the departure date will be 2024-01-01
        designator: 4th group (FBTD-ITI) → Leg 1: 11th field | Leg 2: 20th field | Leg 3: 29th field | Leg 4: 38th field
        stopOver: from 4th group (STPO-ITI) → Leg 1: 3rd field | Leg 2: 12th field | Leg 3: 21st field | Leg 4: 30th field
        flyerCode: <empty>
        fare: you need to check the fare construction group and then apply the same logic as CAT loader (use the same rules applied on these stories: FTS-1518, FTS-1188 item 2, FTS-1188 and FTS-1502)
        currency: same as CURRENCY_CODE
        originalFare: you need to check the fare construction group and then apply the same logic as CAT loader (use the same rules applied on these stories: FTS-1518 and FTS-1188 item 2)
        originalCurrency: same as ORIGINAL_CURRENCY'''

    # FAR
    # fare
    FRCA = json.loads(fare_construction)[0]['content']
    # split
    FAR_split = re.findall(r'[A-PR-Z][\d\.]+', FRCA.split("END")[0])  # filter startswith 'Q'
    # fare_legs, fare_total
    if len(FAR_split) > 0:
        FAR_split = [x[1:] for x in FAR_split]  # filter start alpha
        if (len(FAR_split) > 1):
            # if two more than two ticket legs, direction from right -> left
            FAR_split.reverse()
            # toal_far
            fare_total = FAR_split[0]
            # leg far
            fare_legs = FAR_split[1:]
        elif (len(FAR_split) == 1):
            fare_total = FAR_split[0]
            fare_legs = -0.99
        else:
            fare_total = -0.999
            fare_legs = -0.99

    # ITI
    legs = []
    if len(ITI_L) > 0:
        ITI_L.reverse()
    for i in range(len(ITI_L)):
        conjunction = ticket_number if len(ITI_L[i]['CJNR']) == 0 else (ticket_number[0:3] + ITI_L[i]['CJNR'])
        currency = currency_code
        originalCurrency = original_currency

        # loop
        elements = ITI_L[i]['ITI_L_L']
        elements.reverse()
        k = 0  # index for fare_legs segment
        for element in elements:
            if (element['ORAC'] != ''):
                # departureOn
                # dep_on_date(issue_year-mm-dd)if dep_on_date(mm-dd) > issue_data(mm-dd) else   dep_on_date((issue_year+1)-mm-dd))
                dep_on_date = element['FTDA']  # 4th group (FTDA-ITI)
                if len(dep_on_date) != 0:
                    if re.match(r'^\d{4}-\d{2}-\d{2}$', str(issue_date)):
                        issue_year = issue_date[0:4]
                        dep_day = datetime.strptime(dep_on_date + '2024', '%d%b%Y').strftime("%m-%d")  # 2024 is leap year to avoid   '02-28', just for transformation, not use it afterward
                        dep_year = str(issue_year) if dep_day >= issue_date[5:] else str(int(issue_year) + 1)
                        departureOn = str(pd.datetime.strptime(dep_on_date + dep_year, '%d%b%Y'))[0:10]
                    else:
                        departureOn = ''
                else:
                    departureOn = ''

                # fare
                # far if 'X' != 0, take one from fare_leg
                if (element['STPO'] == 'X'):
                    fare = str('0.00')
                else:
                    if (fare_legs == 0):
                        fare = 'NULL'
                    elif (k < len(fare_legs)):
                        fare = fare_legs[k]
                        k = k + 1
                    else:
                        fare = 'NULL'

                originalFare = fare

                legs_dict = {
                    "departure": element["ORAC"],  # from 4th group (ORAC-ITI)
                    "destination": element["DSTC"],  # from 4th group (DSTC-ITI)
                    "seatClass": element["CLSC"],  # 4th group (CLSC-ITI)
                    "conjunction": conjunction,
                    "carrier": element["CARR"],  # from 4th group (CARR-ITI)
                    "tripCode": element["FTNR"],  # 4th group (FTNR-ITI)
                    "departureOn": departureOn,
                    "designator": element["FBTD"],  # 4th group (FBTD-ITI)
                    "stopOver": element["STPO"],  # from 4th group (STPO-ITI)
                    "flyerCode": "",  # <empty>
                    "fare": fare,
                    "currency": currency,
                    "originalFare": originalFare,
                    "originalCurrency": originalCurrency
                }
                # append list
                legs.append(legs_dict)
        # reverse
        legs.reverse()

    # data validation
    # check fare total
    leg_fare = '%.2f'%sum([float(x[9:]) for x in re.findall(r'\"fare\"\:\s\"\d+\.\d+', json.dumps(legs))])
    print('leg_fare')
    print(json.dumps(legs))
    print(re.findall(r'\"fare\"\:\d+\.\d+', json.dumps(legs)))
    print(leg_fare)
    exception = check_Match(leg_fare, 'leg_fare', fare_total, 'fare_total', exception)
    #check legs match fare
    h = len(fare_legs) if fare_legs != 0 else 0
    exception = check_Match(h, 'leg', k, 'fare', exception)

    return legs, exception

# fare_construction json def
def fare_construction_json(FAR_json):
    '''FARE_CONSTRUCTION:
            Note: the FAR group might have a loop, that’s why we need to use sequence 1, sequence 2, etc.
            JSON. Example: [{"sequence":1,"content":"AX373911153791006*0626/ 122948"},{"sequence":2,"content":"YHZ PD YMQ54.56CAD54.56END"}]
            content: 1st field from 5th group (FRCA-FAR)'''
    FAR_L = FAR_json['FAR_L']
    # fare_construction
    fare_construction = []
    for i in range(len(FAR_L)):
        # dict
        fare_construction_dict = {
            "sequence" : i + 1,
            "content" : FAR_L[i]['FRCA']
        }

        # append list
        fare_construction.append(fare_construction_dict)

    return json.dumps(fare_construction)

# payment json def
def payment_json(column_def, currency_code, FOP_json = None, EXC_json = None, RFT_json = None):
    # PAT PAX
    if column_def in ('PAT', 'PAX'):
        FOP_L = FOP_json["FOP_L"]
        payment = []
        # PAT
        '''PAT PAYMENT:
            JSON. Example: [{"mode":"CC","type":"CCXX","amount":0.00,"accountNumber":"","approvalCode":"","invoiceNumber":"","currency":"CAD"}]
            mode: 1st field of the loop from 7th group (FPTP-FOP). Use only the first 2 chars
            type: 1st field of the loop from 7th group (FPTP-FOP)
            amount: 6th field of the loop from 7th group (FPAM-FOP)
            accountNumber: 2nd field of the loop from 7th group (FPAC-FOP)
            approvalCode: 5th field of the loop from 7th group (APLC-FOP)
            invoiceNumber: <empty>
            currency: same as CURRENCY_CODE'''
        if column_def == 'PAT':
            for i in range(len(FOP_L)):
                # dict
                payment_dict = {
                    "mode" : FOP_L[i]["FPTP"][:2],
                    "type" : FOP_L[i]["FPTP"],
                    "amount" : FOP_L[i]['FPAM'],
                    "accountNumber" : FOP_L[i]["FPAC"],
                    "approvalCode" : FOP_L[i]["APLC"],
                    "invoiceNumber" : "",
                    "currency" : currency_code
                }

                payment.append(payment_dict)

        # PAX
        '''PAX payment (json)
            IF FPTP (Form of Payment Type) field is NOT empty
            regular json object {} follows the same rules indicated in PAT
            JSON. Example: [{"mode":"CC","type":"CCXX","amount":0.00,"accountNumber":"","approvalCode":"","invoiceNumber":"","currency":"CAD"}]

            IF FPTP (Form of Payment Type) field is empty
            a json object as below
            Example: {"mode":"EX","type":"EX","amount":0.00,"accountNumber":"451123456789001","approvalCode":"","invoiceNumber":"","currency":"USD"}
            mode:  EX
            type: EX
            amount: take the FPAM (Form of Payment AmounT) field from FOP section
            accountNumber: EXC GROUP : 8th field (3 digit)(RACN - EXC) + 9th field (10 digit)(RDNR - EXC) + 10th field (1 digit)(CDGT - EXC) + 22nd field (RCPN - EXC) requirements confirmed
            approvalCode: leave empty
            invoiceNumber: leave empty
            currency: same as CURRENCY_CODE'''
        if column_def == 'PAX':
            for i in range(len(FOP_L)):
                # dict
                payment_dict = {
                    "mode" : FOP_L[i]["FPTP"][:2] if FOP_L[i]["FPTP"][:2] != '' else 'EX',
                    "type" : FOP_L[i]["FPTP"] if FOP_L[i]["FPTP"][:2] != '' else 'EX',
                    "amount" : FOP_L[i]['FPAM'],
                    "accountNumber" : FOP_L[i]["FPAC"],
                    "approvalCode" : EXC_json['RACN'] + EXC_json['RDNR'] + EXC_json['CDGT'] + EXC_json['RCPN'],
                    "invoiceNumber" : "",
                    "currency" : currency_code
                }

                payment.append(payment_dict)

    # RFT
    '''REF payment:
        JSON. Example: [{"mode":"CC","type":"CCVI4000","amount":0.00,"accountNumber":"VI************5960","approvalCode":"","invoiceNumber":"","currency":"USD"}]
        mode: 30th field from REF: use the first 2 characters only (FPTP - REF)
        type: 30th field from REF (FPTP - REF)
        amount: 26th field from REF (AMDU - REF)
        accountnumber: 31th field from （FPAC - REF)
        approvalcode: leave empty
        invoicenumber: leave empty
        currency: USD'''
    if column_def == 'RFT':
        payment = []
        # dict
        payment_dict = {
            "mode" : RFT_json['FPTP'][0:2],
            "type" : RFT_json['FPTP'],
            "amount" : RFT_json['AMDU'],
            "accountNumber" : RFT_json['FPAC'],
            "approvalCode" : "",
            "invoiceNumber" : "",
            "currency" : currency_code
            }

        payment.append(payment_dict)

    return json.dumps(payment)

# refund_legs json def
def refund_legs_json(RFT_json):
    '''REFUND_LEGS:
        note: to be populated if ticket_type is RFND only
        JSON. example: [{"sequence":1,"ticketNumber":"0011259634355","coupons":"1000","issueDate":"2022-05-03"}]
        ticketNumber: 17th field + 18th fied from 2nd group (RACN-REF+ RDNR-REF). Note: it may have a loop here, so you need to use the sequence 1, sequence 2, etc in the JSON
        issueDate: 20th fied from 2nd group (ODOI-REF).
        coupons: 19th fied from 2nd group (RCPN-REF). Note: we receive this field as chars. You need to convert into numbers, use the same logic as CAT Loader (for example, the field comes as RR, you need to convert into “1200”. Or might come as VRRV, for example, you need to convert into “0230”. You need to apply the number/sequence only for the letter R; to the other letters (or blank) you need to put “0”)'''
    RFT_L = RFT_json['RFT_L']
    issueDate = RFT_json['ODOI']
    refund_legs = []
    for i in range(len(RFT_L)):
        # dict
        refund_legs_dict = {
            "sequence" : i + 1,
            "ticketNumber" : RFT_L[i]['RACN'] + RFT_L[i]['RDNR'],
            "coupons" : coupons_format(RFT_L[i]['RCPN']),
            "issueDate" : issueDate
        }

        # append list
        refund_legs.append(refund_legs_dict)

    return json.dumps(refund_legs)

In [64]:
# BOS
class BOS:
    def __init__(self, line, currency_code, bos_type):
        self.line = line
        self.currency_code = currency_code
        self.bos_type = bos_type

    # split
    def split_DCI(self):
        # init
        DCI_json = None
        # split
        split1 = self.line.split("")

        # DCI
        DCI = split1[0].split("<")[0]
        if DCI is not None:
            # split
            DCI_split = DCI.split("")

            # json
            DCI_json = {
                'VLNC': DCI_split[1],
                'BACN': DCI_split[4],
                'BDNR': DCI_split[5],
                'DAIS': DCI_split[7]
            }

        return DCI_json

    # map
    def map_DCI(self, exception, DCI_json):
        # init
        agency_code, ticket_number, issue_date = None, None, None
        #check None
        exception = check_None(DCI_json, 'DCI', exception)

        print('DCI_json')
        print(DCI_json)
        print(DCI_json["VLNC"])

        if len(exception) == 0:
            # DCI
            # agency_code : 2nd field from 1st group (VLNC-DCI)
            agency_code = DCI_json['VLNC']
            # ticket_number : 5th field + 6th field from 1st group (BACN-DCI+ BDNR-DCI)
            ticket_number = DCI_json['BACN'] + DCI_json['BDNR']
            # issue_date : 8th field from 1st group (DAIS-DCI). Field is reported as YYMMDD
            issue_date = datetime.strptime(DCI_json['DAIS'], '%d%b%y').strftime("%Y-%m-%d")
            print('DCI_Map')
            print(ticket_number)

        return exception, agency_code, ticket_number, issue_date

    # parser
    def parser(self):
        pass

# PAT
class PATX(BOS):

    # split
    def split_PATX(self):
        DCI_json, SAL_json, TAX_json, ITI_json, FAR_json, FOP_json, EXC_json = None, None, None, None, None, None, None

        # split
        split1 = self.line.split("")

        # DCI
        DCI_json = self.split_DCI()

        # SAL
        SAL = split1[0].split("<")[1]
        if SAL is not None:
            # split
            SAL_split = SAL.split("")

            # json
            SAL_json = {
                'CPUI': SAL_split[6],
                'TDAM': SAL_split[7],
                'FARE': SAL_split[8],
                'CUOF': SAL_split[9],
                'EQFR': SAL_split[10],
                'TTAX': SAL_split[11],
                'COAM': SAL_split[12],
                'CORT': SAL_split[13],
                'PNRR': SAL_split[14],
                'PXNM': SAL_split[15],
                'TOUR': SAL_split[16]
            }

        # TAX
        TAX = split1[1]
        if TAX is not None:
            # split
            TAX_split = TAX.split('<')[0].replace('', '').split('<')

            # vars
            # TAX_L1 : TMFA, TMFT
            TAX_L1 = list()
            for i in range(len(TAX_split)):
                # split
                element = TAX_split[i].split('')

                # dict
                dict = {
                    'TMFA': element[0],
                    'TMFT': element[1]
                }

                # append list
                TAX_L1.append(dict)

            # json
            TAX_json = {
                'TAX_L1': TAX_L1
            }

        # ITI
        ITI = split1[2]
        if ITI is not None:
            # split
            ITI_split = ITI.replace('', '').replace('<', '').split('<')

            # vars
            ITI_L = list()
            for i in range(len(ITI_split)):
                # split
                element = ITI_split[i].split('')

                # vars
                CJNR = element[0]
                CDGT = element[1]

                # segment list
                ITI_L_L = list()
                # dict
                # 4 segment
                for j in range(4):
                    segment_dict = {
                        'STPO': element[(j * 9) + 2],
                        'ORAC': element[(j * 9) + 3],
                        'DSTC': element[(j * 9) + 4],
                        'CARR': element[(j * 9) + 5],
                        'FTNR': element[(j * 9) + 6],
                        'CLSC': element[(j * 9) + 7],
                        'FTDA': element[(j * 9) + 8],
                        'FTDT': element[(j * 9) + 9],
                        'FBTD': element[(j * 9) + 10]
                    }

                    # append segment list
                    ITI_L_L.append(segment_dict)

                # ITI_dict
                ITI_dict = {
                    'CJNR': CJNR,
                    'CDGT': CDGT,
                    'ITI_L_L': ITI_L_L
                }

                # append list
                ITI_L.append(ITI_dict)

            # json
            ITI_json = {
                'ITI_L': ITI_L
            }

        # FAR
        FAR = split1[3]
        if FAR is not None:
            # split
            FAR_split = FAR.replace('', '').replace('<', '').split('<')

            # vars
            # FAR_L
            FAR_L = []
            for i in range(len(FAR_split)):
                # split
                element = FAR_split[i].split('')

                # dict
                FAR_dict = {
                    'FRCA': element[0]
                }

                # append list
                FAR_L.append(FAR_dict)

            # json
            FAR_json = {
                'FAR_L': FAR_L
            }

        # FOP
        FOP = split1[4]
        if FOP is not None:
            # split
            FOP_split = FOP.replace('', '').replace('<', '').split('<')

            # vars
            FOP_L = []
            for i in range(len(FOP_split)):
                # split
                FOP_element = FOP_split[i].split("")

                # dict
                FOP_dict = {
                    'FPTP': FOP_element[0],
                    'FPAC': FOP_element[1],
                    'APLC': FOP_element[4],
                    'FPAM': FOP_element[5]
                }

                # append list
                FOP_L.append(FOP_dict)

            # json
            FOP_json = {
                'FOP_L': FOP_L
            }

        # EXC
        if self.bos_type == 'PAX':
            EXC = split1[7]
            if EXC is not None:
                # split
                EXC_split = EXC.split('<')
                EXC2_split = EXC_split[1].replace('', '').split('<')
                EXC2_element = EXC2_split[0].split("")

                # json
                EXC_json = {
                    'RACN': EXC2_element[0],
                    'RDNR': EXC2_element[1],
                    'CDGT' : EXC2_element[2],
                    'RCPN' : EXC2_element[13]
                }


        return DCI_json, SAL_json, TAX_json, ITI_json, FAR_json, FOP_json, EXC_json

    # map
    def map_PATX(self, exception, DCI_json, SAL_json, TAX_json, ITI_json, FAR_json, FOP_json, EXC_json):
        # init
        exception, agency_code, ticket_number, issue_date, pnr, tour_code, passenger_name, coupon_used, original_fare, fare_amount, exchange_rate, commission_amount, original_currency, tax_amount, total_amount, commission, tax, fare_construction, legs, payment, org_ticket_no = [], None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
        # DCI
        exception, agency_code, ticket_number, issue_date = self.map_DCI(exception, DCI_json)
        #check None
        exception = check_None(DCI_json, 'DCI', exception)
        exception = check_None(SAL_json, 'SAL', exception)
        exception = check_None(TAX_json, 'TAX', exception)
        exception = check_None(ITI_json, 'ITI', exception)
        exception = check_None(FAR_json, 'FAR', exception)
        exception = check_None(FOP_json, 'FOP', exception)
        if len(exception) == 0:
            # vars
            pnr = SAL_json['PNRR']  # 15th field from 2nd group (PNRR-SAL)
            tour_code = SAL_json['TOUR']  # 17th field from 2nd group (TOUR-SAL)
            passenger_name = SAL_json['PXNM'] # 16th field from 2nd group (PXNM-SAL)
            coupon_used = SAL_json['CPUI'] # 7th field from 2nd group (CPUI-SAL)
            original_fare = SAL_json['FARE'] # 9th field from 2nd group(FARE-SAL)
            original_currency = SAL_json['CUOF']  # 10th field from 2nd group (CUOF-SAL)
            tax_amount = SAL_json['TTAX']  # 12th field from 2nd group (TTAX-SAL)
            total_amount = SAL_json['TDAM']  # 8th field from 2nd group (TDAM-SAL)
            fare_amount = SAL_json['EQFR'] if float(SAL_json['EQFR']) != 0 else original_fare  # fare_amount : 11tfrom 2nd group (EQFR-SAL)… Note: if is 0.00, use the same as ORIGINAL_FARE
            exchange_rate = round(float(fare_amount) / float(original_fare), 3) if float(original_fare) != 0 else None #FARE_AMOUNT / ORIGINAL_FARE
            commission_amount = fare_amount  # commission_amount : same as FARE_AMOUNT
            commission = commission_json(SAL_json, self.currency_code)  #JSON. E[{"type":"BASE","amount":4.26,"currency":"CAD","commissionRate
            tax = tax_json(TAX_json['TAX_L1'], original_currency)  # JSON. E[{"type":"CA","amount":7.12,"currency":"CAD"},{"type":"YR","amount":16.00,"currency":
            fare_construction = fare_construction_json(FAR_json) # JSON. E[{"sequence":1,"content":"AX373911153791006*0626/ 122948"},{"sequence":2,"content":YMQ54.56CAD54.5
            legs, exception = legs_json(fare_construction, ITI_json['ITI_L'], self.currency_code, original_currency,ticket_number,  issue_date, exception)  # JSON. example: [{"departure":"FLR","destination":"YYZ","seatC","conjunction":"1111234567890","carrier":"AC","tripCode":"876","departureOn":"2022-12-30","designatoropOver":"X","flyerCode":"","fare":259.25,"currency": "CAD","originalFare":259.25,


            # "originalCurrency"
            if (self.bos_type == 'PAT'):
                # pament
                payment = payment_json(self.bos_type, self.currency_code, FOP_json = FOP_json) # JSON. Example: [{"mode":"CC","type":"CCXX","amount":0.00,"accountNumber":"","approvalCode":"","invoiceNumber":"","currency":"CAD"}]
                org_ticket_no = ticket_number # same as TICKET_NUMBER

            if (self.bos_type == 'PAX'):
                exception = check_None(EXC_json, 'EXC', exception)
                if len(exception) == 0:
                    # pament
                    payment = payment_json(self.bos_type, self.currency_code, FOP_json = FOP_json, EXC_json = EXC_json) # JSON. Example: [{"mode":"EX","type":"EX","amount":0.00,"accountNumber":"","approvalCode":"","invoiceNumber":"","currency":"CAD"}]
                    # org_ticket_no
                    org_ticket_no = EXC_json['RACN'] + EXC_json['RDNR'] # 8th field from EXC (3 digit)(RACN - EXC) + 9th field from EXC (10 digit)(RDNR - EXC) – this is the field that indicates the originating ticket subjected to exchange

            # Data Validation
            # original_fare
            original_fare, exception = check_float(original_fare, 'original_fare', exception, 12, 5)
            # exchange_rate
            exchange_rate, exception = check_float(exchange_rate, 'exchange_rate', exception, 12, 5)
            # fare_amount
            fare_amount, exception = check_float(fare_amount, 'fare_amount', exception, 12, 5)
            # tax_amount
            tax_amount, exception = check_float(tax_amount, 'tax_amount', exception, 12, 5)
            # total_amount
            total_amount, exception = check_float(total_amount, 'total_amount', exception, 12, 5)
            # coupon_used
            (coupon_used, exception) = check_len(coupon_used, 'coupon_used', exception, 0, 10)

        return exception, agency_code, ticket_number, issue_date, pnr, tour_code, passenger_name, coupon_used, original_fare, fare_amount, exchange_rate, commission_amount, original_currency, tax_amount, total_amount, commission, tax, fare_construction, legs, payment, org_ticket_no

    # parser
    def parser(self):
        # init
        exception, agency_code, ticket_number, issue_date, pnr, tour_code, passenger_name, coupon_used, original_fare, fare_amount, exchange_rate, commission_amount, original_currency, tax_amount, total_amount, commission, tax, fare_construction, legs, payment, org_ticket_no, ticket_type = [], None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None

        if self.bos_type == 'PAT':
            ticket_type = 'TKTT'
        elif self.bos_type == 'PAX':
            ticket_type = 'EXCH-TKTT'

        # Parser
        DCI_json, SAL_json, TAX_json, ITI_json, FAR_json, FOP_json, EXC_json = self.split_PATX()

        # Map
        exception, agency_code, ticket_number, issue_date, pnr, tour_code, passenger_name, coupon_used, original_fare, fare_amount, exchange_rate, commission_amount, original_currency, tax_amount, total_amount, commission, tax, fare_construction, legs, payment, org_ticket_no =self.map_PATX(exception, DCI_json, SAL_json, TAX_json, ITI_json, FAR_json, FOP_json, EXC_json)

        return exception, agency_code, ticket_number, issue_date, pnr, tour_code, passenger_name, coupon_used, original_fare, fare_amount, exchange_rate, commission_amount, original_currency, tax_amount, total_amount, commission, tax, fare_construction, legs, payment, org_ticket_no, ticket_type

# RFT
class RFT(BOS):
    def split_RFT(self):
        DCI_json, RFT_json = None, None

        # DCI
        DCI_json = self.split_DCI()

        # RFT
        RFT1 = re.search(r'<.+', self.line).group()
        print('RFT')
        print(RFT)
        if RFT1 is not None:
            # split
            RFT_split = RFT1.split('<')
            RFT_split_1 = RFT_split[0].split('')[1].split('<')
            RFT_split_2_element = RFT_split[1].split("")

            # RFT_L
            RFT_L1 = []
            # element
            for i in range(len(RFT_split_1)):
                element = RFT_split_1[i].split("")

                # dict
                RFT_dict = {
                    'RACN': element[0],
                    'RDNR': element[1],
                    'RCPN': element[2]
                }

                # append list
                RFT_L1.append(RFT_dict)

            # json
            RFT_json = {
                'ODOI' : RFT_split_2_element[0],
                'AMDU' : RFT_split_2_element[3],
                'FPTP': RFT_split_2_element[8],
                'FPAC': RFT_split_2_element[9],
                'RFT_L': RFT_L1
            }

        return DCI_json, RFT_json

    def map_RFT(self, exception, DCI_json, RFT_json):
        # init
        exception, agency_code, ticket_number, issue_date, payment, refund_legs, org_ticket_no, ticket_type = [], None, None, None, None, None, None, None
        # DCI
        exception, agency_code, ticket_number, issue_date = self.map_DCI(exception, DCI_json)
        #check None
        exception = check_None(DCI_json, 'DCI', exception)
        exception = check_None(RFT_json, 'EXC', exception)
        if len(exception) == 0:
            # vars
            # payment
            payment = payment_json(self.bos_type, self.currency_code, RFT_json = RFT_json) # JSON. Example: [{"mode":"CC","type":"CCVI4000","amount":0.00,"accountNumber":"VI************5960","approvalCode":"","invoiceNumber":"","currency":"USD"}]

            # refund_legs
            refund_legs = refund_legs_json(RFT_json) # JSON. example: [{"sequence":1,"ticketNumber":"0011259634355","coupons":"1000","issueDate":"2022-05-03"}]

            # org_ticket_no
            org_ticket_no = RFT_json['RFT_L'][0]['RACN'] + RFT_json['RFT_L'][0]['RDNR'] # same as 1st ticketNumber from REFUND_LEGS

        print('map_RFT')
        print(ticket_number)

        return exception, agency_code, ticket_number, issue_date, payment, refund_legs, org_ticket_no

    def parser(self):
        # init
        exception, agency_code, ticket_number, issue_date, pnr, tour_code, passenger_name, coupon_used, original_fare, fare_amount, exchange_rate, commission_amount, original_currency, tax_amount, total_amount, commission, tax, fare_construction, legs, payment, refund_legs, org_ticket_no = [], None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None

        ticket_type = 'RFND'

        # Parser
        DCI_json, RFT_json = self.split_RFT()

        # Map
        exception, agency_code, ticket_number, issue_date, payment, refund_legs, org_ticket_no =self.map_RFT(exception, DCI_json, RFT_json)
        print('ticket_number')
        print(ticket_number)

        return exception, agency_code, ticket_number, issue_date, payment, refund_legs, org_ticket_no, ticket_type

def BOS_parser(fn : BOS):
    return fn.parser()

def br_map(line):
    # init
    exception, agency_code, ticket_number, issue_date, pnr, tour_code, passenger_name, coupon_used, original_fare, fare_amount, exchange_rate, commission_amount, original_currency, tax_amount, total_amount, commission, tax, fare_construction, legs, payment, refund_legs, org_ticket_no, ticket_type = [], None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None

    currency_code = "USD"

    # split bos type
    bos_type = line.split("")[0]

    # File Parser
    if bos_type in ("PAT", "PAX", "RFT"):
        if (bos_type in ("PAT", "PAX")):
            bos_file_type = PATX(line, currency_code, bos_type)

            # Parser
            exception, agency_code, ticket_number, issue_date, pnr, tour_code, passenger_name, coupon_used, original_fare, fare_amount, exchange_rate, commission_amount, original_currency, tax_amount, total_amount, commission, tax, fare_construction, legs, payment, org_ticket_no, ticket_type = BOS_parser(bos_file_type)
        elif (bos_type == 'RFT'):
            bos_file_type = RFT(line, currency_code, bos_type)
            exception, agency_code, ticket_number, issue_date, payment, refund_legs, org_ticket_no, ticket_type = BOS_parser(bos_file_type)

    else:
        ticket_type = bos_type

    return {
        "exception" : exception,  \
        "source" : "BOS", \
        "booking_channel" : "WEB", \
        "version_no" : 1, \
        "ticket_type" : ticket_type, \
        "country_code" : "USA",\
        "passenger_count" : 1,
        "currency_code" : currency_code, \
        "org_ticket_no" : org_ticket_no, \
        "agency_code" : agency_code, \
        "ticket_number" : ticket_number, \
        "issue_date" : issue_date, \
        "pnr" : pnr, \
        "tour_code" : tour_code, \
        "passenger_name" : passenger_name, \
        "coupon_used" : coupon_used, \
        "original_fare" : original_fare, \
        "fare_amount" : fare_amount, \
        "exchange_rate" : exchange_rate, \
        "commission_amount" : commission_amount, \
        "original_currency" : original_currency, \
        "tax_amount" : tax_amount, \
        "total_amount" : total_amount, \
        "commission" : commission, \
        "tax" : tax, \
        "fare_construction" : fare_construction, \
        "legs" : legs, \
        "payment" : payment, \
        "refund_legs": refund_legs,\
        "pos" : "1"
    }

# schema
schema = StructType([ \
    StructField("exception", StringType(), True), \
    StructField("source", StringType(), True), \
    StructField("booking_channel", StringType(), True), \
    StructField("version_no", IntegerType(), True), \
    StructField("ticket_type", StringType(), True), \
    StructField("currency_code", StringType(), True), \
    StructField("org_ticket_no", StringType(), True), \
    StructField("agency_code",StringType(),True), \
    StructField("ticket_number",StringType(),True), \
    StructField("issue_date",StringType(),True), \
    StructField("country_code",StringType(),True), \
    StructField("passenger_count",IntegerType(),True), \
    StructField("pnr", StringType(), True), \
    StructField("tour_code", StringType(), True), \
    StructField("passenger_name", StringType(), True), \
    StructField("coupon_used", StringType(), True), \
    StructField("original_fare", DoubleType(), True), \
    StructField("fare_amount", DoubleType(), True), \
    StructField("exchange_rate", DoubleType(), True), \
    StructField("commission_amount", DoubleType(), True), \
    StructField("original_currency", IntegerType(), True), \
    StructField("tax_amount", DoubleType(), True), \
    StructField("total_amount", DoubleType(), True), \
    StructField("commission", StringType(), True), \
    StructField("tax", StringType(), True), \
    StructField("fare_construction", StringType(), True), \
    StructField("legs", StringType(), True), \
    StructField("payment", StringType(), True), \
    StructField("refund_legs", StringType(), True),\
    StructField("pos", StringType(), True)
  ])

df = sparkDF.withColumn("bre_list",udf(br_map, schema)("BOS_FILE_EXTRACT"))
df = df.withColumns({
    "exception": df["bre_list"]["exception"],\
    "source": df["bre_list"]["source"],\
    "booking_channel": df["bre_list"]["booking_channel"],\
    "version_no": df["bre_list"]["version_no"],\
    "ticket_type": df["bre_list"]["ticket_type"],\
    "currency_code": df["bre_list"]["currency_code"],\
    "org_ticket_no": df["bre_list"]["org_ticket_no"],\
    "agency_code": df["bre_list"]["agency_code"],\
    "ticket_number": df["bre_list"]["ticket_number"],\
    "issue_date": df["bre_list"]["issue_date"],\
    "pnr": df["bre_list"]["pnr"],\
    "tour_code": df["bre_list"]["tour_code"],\
    "passenger_name": df["bre_list"]["passenger_name"],\
    "coupon_used": df["bre_list"]["coupon_used"],\
    "original_fare": df["bre_list"]["original_fare"],\
    "fare_amount": df["bre_list"]["fare_amount"],\
    "exchange_rate": df["bre_list"]["exchange_rate"],\
    "commission_amount": df["bre_list"]["commission_amount"],\
    "original_currency": df["bre_list"]["original_currency"],\
    "tax_amount": df["bre_list"]["tax_amount"],\
    "total_amount": df["bre_list"]["total_amount"],\
    "commission": df["bre_list"]["commission"],\
    "tax": df["bre_list"]["tax"],\
    "fare_construction": df["bre_list"]["fare_construction"],\
    "legs": df["bre_list"]["legs"],\
    "payment": df["bre_list"]["payment"],\
    "refund_legs": df["bre_list"]["refund_legs"],\
    "pos": df["bre_list"]["pos"]
})

In [65]:
df.select('bre_list').head(3)

RFTage 14:>                                                         (0 + 1) / 1]
<class '__main__.RFT'>
DCI_json
{'VLNC': '22521623', 'BACN': '125', 'BDNR': '2161454822', 'DAIS': '17JUN23'}
22521623
DCI_Map
1252161454822
map_RFT
1252161454822
ticket_number
1252161454822
DCI_json
{'VLNC': '22521623', 'BACN': '001', 'BDNR': '7976703339', 'DAIS': '17JUN23'}
22521623
DCI_Map
0017976703339
/var/folders/d9/871p0v1d2cn4c6y7bj_xghhh0000gn/T/ipykernel_2961/3275215194.py:113: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
leg_fare
[{"departure": "ORF", "destination": "DCA", "seatClass": "S", "conjunction": "0017976703339", "carrier": "AA", "tripCode": "5508", "departureOn": "2023-07-08", "designator": "SUAIZNN1", "stopOver": "X", "flyerCode": "", "fare": "0.00", "currency": "USD", "originalFare": "0.00", "originalCurrency": "USD"}, {"departure": "DCA", "destination": "GRR", "seatClass": "S", "conjun

[Row(bre_list=Row(exception='[]', source='BOS', booking_channel='WEB', version_no=1, ticket_type='RFND', currency_code='USD', org_ticket_no='1252161454822', agency_code='22521623', ticket_number='1252161454822', issue_date='2023-06-17', country_code='USA', passenger_count=1, pnr=None, tour_code=None, passenger_name=None, coupon_used=None, original_fare=None, fare_amount=None, exchange_rate=None, commission_amount=None, original_currency=None, tax_amount=None, total_amount=None, commission=None, tax=None, fare_construction=None, legs=None, payment='[{"mode": "CA", "type": "CA", "amount": "-953.65", "accountNumber": "CASH", "approvalCode": "", "invoiceNumber": "", "currency": "USD"}]', refund_legs='[{"sequence": 1, "ticketNumber": "1252161454822", "coupons": "1234", "issueDate": ""}]', pos='1')),
 Row(bre_list=Row(exception='[]', source='BOS', booking_channel='WEB', version_no=1, ticket_type='EXCH-TKTT', currency_code='USD', org_ticket_no='0017965804275', agency_code='22521623', ticket_n

In [58]:
s = '[{"departure": "AMS", "destination": "MAD", "seatClass": "Q", "conjunction": "0742100377105", "carrier": "KL", "tripCode": "1701", "departureOn": "2023-06-24", "designator": "QH5UA5LG/XX", "stopOver": "O", "flyerCode": "", "fare": "119.12", "currency": "USD", "originalFare": "119.12", "originalCurrency": "USD"}, {"departure": "MAD", "destination": "AMS", "seatClass": "E", "conjunction": "0742100377105", "carrier": "KL", "tripCode": "1702", "departureOn": "2023-06-27", "designator": "EH5UA5LG", "stopOver": "O", "flyerCode": "", "fare": "97.52", "currency": "USD", "originalFare": "97.52", "originalCurrency": "USD"}]'

re.findall(r'\"fare\"\:\s\"\d+\.\d+', s)[0][9:]

'119.12'

In [69]:
s = 'RFT2252162320230618230618050933261252161454822417JUN23IARBA/D/ALLRECORDS/18JUN23103<953.650.000.00358.00595.650.000.000.000.0012521614548221234<REDACTED REF NAMEF-953.650.000.00-953.65CACASH<'
re.search(r'<.+', s).group()

'<\x80\x80\x80\x80\x80953.65\x800.00\x800.00\x80358.00\x80595.65\x800.00\x800.00\x800.00\x800.00\x80\x80\x80\x81125\x802161454822\x801234\x80<\x82\x80REDACTED REF NAME\x80F\x80-953.65\x800.00\x800.00\x80\x80-953.65\x80CA\x80CASH\x80<'

In [34]:
k1 = sparkDF.withColumn('test', F.lit([{'123':789},{'456':890}]))

In [54]:
k2 = df.withColumns({'a':df['bre_list']['num'],'b':df['bre_list']['letters']})

In [55]:
k2.select('a','b').show()

+---+---+
|  a|  b|
+---+---+
|123|456|
|123|456|
|123|456|
+---+---+



In [47]:
k3 = k1.withColumns({'a':k1.test['123'],'b':k1.test['456']})

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "test[123]" due to data type mismatch: Parameter 2 requires the "INTEGRAL" type, however "123" has the type "STRING".;
'Project [ROW_ID#12L, file_path#13, BOS_FILE_EXTRACT#14, COLUMN_DEF#15, test#26, test#26[123] AS a#94, test#26[456] AS b#95]
+- Project [ROW_ID#12L, file_path#13, BOS_FILE_EXTRACT#14, COLUMN_DEF#15, array(123, 456) AS test#26]
   +- LogicalRDD [ROW_ID#12L, file_path#13, BOS_FILE_EXTRACT#14, COLUMN_DEF#15], false


In [41]:
k2.select('a','b').show()

+---+---+
|  a|  b|
+---+---+
|123|456|
|123|456|
|123|456|
+---+---+

